In [1]:
import pandas as pd
import spacy

gpu = spacy.prefer_gpu()

In [2]:
dados_treino = pd.read_csv("../introducao-word-embedding/treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
72389,'Cinco Graças' se esvazia quando trata conflit...,O cinema contemporâneo às vezes parece com aqu...,2016-02-18,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/02...
55520,'TV Folha' tira dúvidas sobre Previdência ao v...,"A ""TV Folha"" responderá ao vivo dúvidas sobre ...",2016-05-31,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/05/1...
65236,Agosto é o 3° mais chuvoso desde 1995 e eleva ...,O mês de agosto é historicamente seco. Neste a...,2016-01-09,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/09...
81347,Biografia falocêntrica mostra vedete Rogéria c...,Uma senhora loira de 73 anos vê uma moto na vi...,2016-10-20,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/10...
23747,"Morto há dois anos, Eduardo Coutinho planejava...","Com futuro incerto, Eduardo Coutinho queria re...",2016-02-02,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2016/02...


In [ ]:
#Rodas apenas uma vez
# !python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 89.9MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186282 sha256=e6686ba775d01478f4e0e215883920b63ad81fd819ef08daf6ceeaba8f527e86
  Stored in directory: /tmp/pip-ephem-wheel-cache-tsi63j7g/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [3]:
nlp = spacy.load("pt_core_news_sm")

In [4]:
texto = "Rio de Janeiro é uma cidade maravilhosa"
doc = nlp(texto)

In [5]:
doc

Rio de Janeiro é uma cidade maravilhosa

In [6]:
type(doc[2])

spacy.tokens.token.Token

In [7]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [8]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [9]:
texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [10]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 1000,
                                                        # n_process = -1
                                                        )]

tf = time() - t0

print(tf/60)

0.3848552028338114


In [11]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [13]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [14]:
w2v_modelo

In [15]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [16]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [18]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2024-06-05 18:33:08,104 : - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2024-06-05T18:33:08.104888', 'gensim': '4.3.0', 'python': '3.11.9 | packaged by conda-forge | (main, Apr 19 2024, 18:36:13) [GCC 12.3.0]', 'platform': 'Linux-6.8.0-35-generic-x86_64-with-glibc2.39', 'event': 'created'}
2024-06-05 18:33:08,105 : - collecting all words and their counts
2024-06-05 18:33:08,105 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-06-05 18:33:08,109 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2024-06-05 18:33:08,112 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2024-06-05 18:33:08,116 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2024-06-05 18:33:08,120 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2024-06-05 18:33:08,124 : - PROGRESS: at sentence #25000, processed 15